In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [21]:
crime_data = pd.read_csv('https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv')
crime_data.head()

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,UPDATEDON,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,08/28/2004 05:50:56 PM,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,02/10/2018 03:50:01 PM,41.807441,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,06/26/2004 12:40:00 PM,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,False,False,1112,11,27.0,23.0,6,1152206.0,1906127.0,2004,02/28/2018 03:56:25 PM,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,04/04/2011 05:45:00 AM,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,221,2,3.0,38.0,6,1177436.0,1876313.0,2011,02/10/2018 03:50:01 PM,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,12/30/2010 04:30:00 PM,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,423,4,7.0,46.0,6,1194622.0,1850125.0,2010,02/10/2018 03:50:01 PM,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,02/02/2016 07:30:00 PM,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,False,False,831,8,15.0,66.0,6,1155240.0,1860661.0,2016,02/10/2018 03:50:01 PM,41.773455,-87.706480,"(41.773455295, -87.706480471)"


In [19]:
census_data = pd.read_csv('https://ibm.box.com/shared/static/05c3415cbfbtfnr2fx4atenb2sd361ze.csv')
census_df = census_data[['COMMUNITY_AREA_NAME','COMMUNITY_AREA_NUMBER']].dropna()
census_df.head()

,COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
0,Rogers Park,1.0
1,West Ridge,2.0
2,Uptown,3.0
3,Lincoln Square,4.0
4,North Center,5.0


In [40]:
merged_df = pd.merge(crime_data,census_df,left_on='COMMUNITY_AREA_NUMBER',right_on='COMMUNITY_AREA_NUMBER').reset_index(drop=True).dropna()
merged_df.head()

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,UPDATEDON,LATITUDE,LONGITUDE,LOCATION,COMMUNITY_AREA_NAME
0,3512276,HK587712,08/28/2004 05:50:56 PM,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,02/10/2018 03:50:01 PM,41.807441,-87.703956,"(41.8074405, -87.703955849)",Brighton Park
1,7937542,HT169840,02/18/2011 07:30:00 PM,029XX W 36TH ST,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE-GARAGE,False,False,913,9,12.0,58.0,14,1157190.0,1880621.0,2011,02/10/2018 03:50:01 PM,41.828189,-87.698792,"(41.828189011, -87.698792003)",Brighton Park
2,7881407,HT112422,01/09/2011 06:27:00 PM,039XX S ROCKWELL ST,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,914,9,12.0,58.0,14,1159685.0,1878264.0,2011,02/04/2016 06:33:39 AM,41.821670,-87.689703,"(41.821670189, -87.689702831)",Brighton Park
3,10770065,HZ535494,11/30/2016 10:00:00 PM,033XX W 38TH ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,911,9,12.0,58.0,14,1154734.0,1879205.0,2016,02/10/2018 03:50:01 PM,41.824353,-87.707841,"(41.824352752, -87.707840716)",Brighton Park
4,7502426,HS305355,05/13/2010 11:00:00 AM,035XX S WASHTENAW AVE,1821,NARCOTICS,MANU/DEL:CANNABIS 10GM OR LESS,"SCHOOL, PUBLIC, BUILDING",True,False,913,9,12.0,58.0,18,1158944.0,1880896.0,2010,02/10/2018 03:50:01 PM,41.828908,-87.692349,"(41.828907913, -87.692349187)",Brighton Park


In [101]:
chicago_df = merged_df[['ID','PRIMARY_TYPE', 'LATITUDE', 'LONGITUDE', 'COMMUNITY_AREA_NAME']]
chicago_df.head()

,ID,PRIMARY_TYPE,LATITUDE,LONGITUDE,COMMUNITY_AREA_NAME
0,3512276,THEFT,41.807441,-87.703956,Brighton Park
1,7937542,CRIMINAL DAMAGE,41.828189,-87.698792,Brighton Park
2,7881407,CRIMINAL DAMAGE,41.821670,-87.689703,Brighton Park
3,10770065,CRIMINAL DAMAGE,41.824353,-87.707841,Brighton Park
4,7502426,NARCOTICS,41.828908,-87.692349,Brighton Park


In [89]:
address = 'Chicago, IL'

geolocator = Nominatim(user_agent="il_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago are 41.8755616, -87.6244212.


In [102]:
# create map of Chicago using latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers of incident locations to map
for lat, lng, community, crime in zip(chicago_df['LATITUDE'], chicago_df['LONGITUDE'], chicago_df['COMMUNITY_AREA_NAME'], chicago_df['PRIMARY_TYPE']):
    label = '{}, {}'.format(community, crime)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

In [150]:
CLIENT_ID = '5BLMRFCXXDZWTGCWEY0WRLEFDWGGYY5BTARJX45C04ZYXPME' # your Foursquare ID
CLIENT_SECRET = 'BZLR4TQY2PIIRZG12ZGDPOTK3PLQCJCDGDCS2A0NWCSALFTZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5BLMRFCXXDZWTGCWEY0WRLEFDWGGYY5BTARJX45C04ZYXPME
CLIENT_SECRET:BZLR4TQY2PIIRZG12ZGDPOTK3PLQCJCDGDCS2A0NWCSALFTZ


In [146]:
chicago_df.loc[0, 'COMMUNITY_AREA_NAME']

'Brighton Park'

In [147]:
community_latitude = chicago_df.loc[0, 'LATITUDE'] # community latitude value
community_longitude = chicago_df.loc[0, 'LONGITUDE'] # community longitude value

community_name = chicago_df.loc[0, 'COMMUNITY_AREA_NAME'] # community name

print('Latitude and longitude values of {} are {}, {}.'.format(community_name, 
                                                               community_latitude, 
                                                               community_longitude))

Latitude and longitude values of Brighton Park are 41.807440500000006, -87.70395585.


In [148]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    community_latitude, 
    community_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=D4L3BAWJD52YRDAGKUYDHCWGN1Y4RRBGDC2WHAJELHTMVJKC&client_secret=P0KEXQ5BZ4FRK5JJRKZF35KVINZZYANN2AUVRA53Y4OGBBRR&v=20180605&ll=41.807440500000006,-87.70395585&radius=500&limit=100'

In [149]:
results = requests.get(url).json()
results

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5e2dbd13b4b684001b9f383b'},
 'response': {}}

In [137]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [138]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pete's Fresh Market,Grocery Store,41.807767,-87.704284
1,Carnitas Don Rafa,Mexican Restaurant,41.809206,-87.703963
2,Subway,Sandwich Place,41.807603,-87.703264
3,Dollar Tree,Discount Store,41.806572,-87.704529
4,T.J. Maxx,Department Store,41.803621,-87.705262


In [139]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

31 venues were returned by Foursquare.


In [151]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Community', 
                  'Community Latitude', 
                  'Community Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [152]:
chicago_venues = getNearbyVenues(names=chicago_df['COMMUNITY_AREA_NAME'],
                                   latitudes=chicago_df['LATITUDE'],
                                   longitudes=chicago_df['LONGITUDE']
                                  )

print(chicago_venues.shape)

Brighton Park
Brighton Park
Brighton Park
Brighton Park
Brighton Park
Brighton Park
Brighton Park
Brighton Park
Brighton Park
Brighton Park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Humboldt park
Grand Boulevard
Grand Boulevard
Grand Boulevard
Grand Boulevard
Grand Boulevard
Grand Boulevard
Grand Boulevard
Grand Boulevard
South Chicago
South Chicago
South Chicago
South Chicago
South Chicago
South Chicago
South Chicago
South Chicago
South Chicago
Chicago Lawn
Chicago Lawn
Chicago Lawn
Chicago Lawn
Chicago Lawn
Chicago Lawn
Chicago Lawn
Chicago Lawn
Chicago Lawn
Chicago Lawn
Chicago Lawn
Chicago Lawn
West Town
West Town
West Town
West Town
West Town
West Town
West Town
West Town
West Town
West Town
West Town
West Town
West Town
Albany Park
Albany Park
Al

In [153]:
chicago_venues.head()

,Community,Community Latitude,Community Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Brighton Park,41.807441,-87.703956,Pete's Fresh Market,41.807767,-87.704284,Grocery Store
1,Brighton Park,41.807441,-87.703956,Carnitas Don Rafa,41.809206,-87.703963,Mexican Restaurant
2,Brighton Park,41.807441,-87.703956,Subway,41.807603,-87.703264,Sandwich Place
3,Brighton Park,41.807441,-87.703956,Dollar Tree,41.806572,-87.704529,Discount Store
4,Brighton Park,41.807441,-87.703956,T.J. Maxx,41.803621,-87.705262,Department Store


In [154]:
chicago_venues.groupby('Community').count()

,Community Latitude,Community Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Community,,,,,,
Albany Park,142,142,142,142,142,142
Ashburn,47,47,47,47,47,47
Auburn Gresham,113,113,113,113,113,113
Austin,354,354,354,354,354,354
Avalon Park,55,55,55,55,55,55
Avondale,225,225,225,225,225,225
Belmont Cragin,93,93,93,93,93,93
Beverly,71,71,71,71,71,71
Bridgeport,18,18,18,18,18,18


In [155]:
print('There are {} uniques categories.'.format(len(chicago_venues['Venue Category'].unique())))

There are 383 uniques categories.


In [156]:
# one hot encoding
chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")

# add community column back to dataframe
chicago_onehot['Community'] = chicago_venues['Community'] 

# move community column to the first column
fixed_columns = [chicago_onehot.columns[-1]] + list(chicago_onehot.columns[:-1])
chicago_onehot = chicago_onehot[fixed_columns]

chicago_onehot.head()

,Community,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Floating Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,P

In [157]:
chicago_grouped = chicago_onehot.groupby('Community').mean().reset_index()
chicago_grouped

,Community,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Floating Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,P

In [159]:
num_top_venues = 5

for hood in chicago_grouped['Community']:
    print("----"+hood+"----")
    temp = chicago_grouped[chicago_grouped['Community'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albany Park----
                venue  freq
0  Mexican Restaurant  0.08
1         Pizza Place  0.04
2                Park  0.04
3              Bakery  0.04
4         Coffee Shop  0.04


----Ashburn----
                venue  freq
0                Park  0.15
1         Pizza Place  0.06
2  Mexican Restaurant  0.04
3      Clothing Store  0.04
4        Home Service  0.04


----Auburn Gresham----
                  venue  freq
0        Discount Store  0.10
1              Pharmacy  0.08
2    Seafood Restaurant  0.06
3  Fast Food Restaurant  0.05
4    Chinese Restaurant  0.05


----Austin----
                  venue  freq
0  Fast Food Restaurant  0.08
1   Fried Chicken Joint  0.06
2                  Park  0.05
3        Sandwich Place  0.05
4        Discount Store  0.05


----Avalon Park----
                  venue  freq
0  Fast Food Restaurant  0.15
1        Sandwich Place  0.07
2          Burger Joint  0.07
3         Grocery Store  0.05
4                 Diner  0.05


----Avondale----
   

In [160]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [162]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Community']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
community_venues_sorted = pd.DataFrame(columns=columns)
community_venues_sorted['Community'] = chicago_grouped['Community']

for ind in np.arange(chicago_grouped.shape[0]):
    community_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chicago_grouped.iloc[ind, :], num_top_venues)

community_venues_sorted.head()

,Community,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Park,Mexican Restaurant,Coffee Shop,Middle Eastern Restaurant,Bakery,Pizza Place,Park,Grocery Store,Ice Cream Shop,Sandwich Place,Asian Restaurant
1,Ashburn,Park,Pizza Place,Intersection,Clothing Store,Mexican Restaurant,Bar,Grocery Store,Cosmetics Shop,Martial Arts Dojo,Home Service
2,Auburn Gresham,Discount Store,Pharmacy,Seafood Restaurant,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Sandwich Place,Lounge,Liquor Store,Pizza Place
3,Austin,Fast Food Restaurant,Fried Chicken Joint,Park,Discount Store,Sandwich Place,BBQ Joint,Grocery Store,Chinese Restaurant,Mexican Restaurant,Donut Shop
4,Avalon Park,Fast Food Restaurant,Burger Joint,Sandwich Place,Diner,Boutique,Grocery Store,Fried Chicken Joint,Pharmacy,Chinese Restaurant,Food


In [165]:
# set number of clusters
kclusters = 5

chicago_grouped_clustering = chicago_grouped.drop('Community', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chicago_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [168]:
# add clustering labels
community_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [180]:
chicago_merged = chicago_df

# merge chicago_merged with chicago_df to add latitude/longitude for each community
chicago_merged = chicago_merged.join(community_venues_sorted.set_index('Community'), on='COMMUNITY_AREA_NAME')

chicago_merged.sort_values('PRIMARY_TYPE').head() # check the last columns!

,ID,PRIMARY_TYPE,LATITUDE,LONGITUDE,COMMUNITY_AREA_NAME,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
467,4607081,ARSON,41.679809,-87.625870,West Pullman,1,Food,Bar,Bus Station,Gym / Fitness Center,Convenience Store,Train Station,Fast Food Restaurant,Historic Site,Bakery,Grocery Store
453,9692995,ARSON,41.653711,-87.609529,Riverdale,4,Park,Fast Food Restaurant,Harbor / Marina,Yoga Studio,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant
396,4681320,ASSAULT,41.755013,-87.665986,Auburn Gresham,1,Discount Store,Pharmacy,Seafood Restaurant,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Sandwich Place,Lounge,Liquor Store,Pizza Place
397,3661130,ASSAULT,41.740476,-87.654683,Auburn Gresham,1,Discount Store,Pharmacy,Seafood Restaurant,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Sandwich Place,Lounge,Liquor Store,Pizza Place
431,10502099,ASSAULT,41.870142,-87.697037,East Garfield Park,1,Discount Store,Pharmacy,Diner,American Restaurant,Supermarket,Liquor Store,Southern / Soul Food Restaurant,Hot Dog Joint,Food,Bus Line


In [179]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, crime, cluster in zip(chicago_merged['LATITUDE'], chicago_merged['LONGITUDE'], chicago_merged['COMMUNITY_AREA_NAME'],chicago_merged['PRIMARY_TYPE'], chicago_merged['Cluster Labels']):
    label = '{}, {}'.format(poi, crime)
    label = folium.Popup(label + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [181]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 0, chicago_merged.columns[[1] + list(range(5, chicago_merged.shape[1]))]]

,PRIMARY_TYPE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,THEFT,0,Coffee Shop,Mexican Restaurant,Bar,Pizza Place,American Restaurant,Yoga Studio,Pharmacy,Flower Shop,Gym,Gym / Fitness Center
62,THEFT,0,Coffee Shop,Mexican Restaurant,Bar,Pizza Place,American Restaurant,Yoga Studio,Pharmacy,Flower Shop,Gym,Gym / Fitness Center
63,THEFT,0,Coffee Shop,Mexican Restaurant,Bar,Pizza Place,American Restaurant,Yoga Studio,Pharmacy,Flower Shop,Gym,Gym / Fitness Center
64,BATTERY,0,Coffee Shop,Mexican Restaurant,Bar,Pizza Place,American Restaurant,Yoga Studio,Pharmacy,Flower Shop,Gym,Gym / Fitness Center
65,BATTERY,0,Coffee Shop,Mexican Restaurant,Bar,Pizza Place,American Restaurant,Yoga Studio,Pharmacy,Flower Shop,Gym,Gym / Fitness Center
66,CRIMINAL DAMAGE,0,Coffee Shop,Mexican Restaurant,Bar,Pizza Place,American Restaurant,Yoga Studio,Pharmacy,Flower Shop,Gym,Gym / Fitness Center
67,CRIMINAL DAMAGE,0,Coffee Shop,Mexican Restaurant,Bar,Pizza Place,American Restaurant,Yoga Studio,Pharmacy,Flower Shop,Gym,Gym / Fitness Center
68,CRIMINAL DAMAGE,0,Coffee Shop,Mexican Restaurant,Bar,Pizza Place,American Restaurant,Yoga Studio,Pharmacy,Flower Shop,Gym,Gym / Fitness Center
69,CRIMINAL DAMAGE,0,Coffee Shop,Mexican Restaurant,Bar,Pizza Place,American Restaurant,Yoga Studio,Pharmacy,Flower Shop,Gym,Gym / Fitness Center
70,OTHER OFFENSE,0,Coffee Shop,Mexican Restaurant,Bar,Pizza Place,American Restaurant,Yoga Studio,Pharmacy,Flower Shop,Gym,Gym / Fitness Center


In [183]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 1, chicago_merged.columns[[1] + list(range(5, chicago_merged.shape[1]))]]

,PRIMARY_TYPE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,THEFT,1,Mexican Restaurant,Grocery Store,Seafood Restaurant,Taco Place,Sandwich Place,Clothing Store,Donut Shop,Hot Dog Joint,Discount Store,Ice Cream Shop
1,CRIMINAL DAMAGE,1,Mexican Restaurant,Grocery Store,Seafood Restaurant,Taco Place,Sandwich Place,Clothing Store,Donut Shop,Hot Dog Joint,Discount Store,Ice Cream Shop
2,CRIMINAL DAMAGE,1,Mexican Restaurant,Grocery Store,Seafood Restaurant,Taco Place,Sandwich Place,Clothing Store,Donut Shop,Hot Dog Joint,Discount Store,Ice Cream Shop
3,CRIMINAL DAMAGE,1,Mexican Restaurant,Grocery Store,Seafood Restaurant,Taco Place,Sandwich Place,Clothing Store,Donut Shop,Hot Dog Joint,Discount Store,Ice Cream Shop
4,NARCOTICS,1,Mexican Restaurant,Grocery Store,Seafood Restaurant,Taco Place,Sandwich Place,Clothing Store,Donut Shop,Hot Dog Joint,Discount Store,Ice Cream Shop
5,NARCOTICS,1,Mexican Restaurant,Grocery Store,Seafood Restaurant,Taco Place,Sandwich Place,Clothing Store,Donut Shop,Hot Dog Joint,Discount Store,Ice Cream Shop
6,NARCOTICS,1,Mexican Restaurant,Grocery Store,Seafood Restaurant,Taco Place,Sandwich Place,Clothing Store,Donut Shop,Hot Dog Joint,Discount Store,Ice Cream Shop
7,OTHER OFFENSE,1,Mexican Restaurant,Grocery Store,Seafood Restaurant,Taco Place,Sandwich Place,Clothing Store,Donut Shop,Hot Dog Joint,Discount Store,Ice Cream Shop
8,MOTOR VEHICLE THEFT,1,Mexican Restaurant,Grocery Store,Seafood Restaurant,Taco Place,Sandwich Place,Clothing Store,Donut Shop,Hot Dog Joint,Discount Store,Ice Cream Shop
9,HOMICIDE,1,Mexican Restaurant,Grocery Store,Seafood Restaurant,Taco Place,Sandwich Place,Clothing Store,Donut Shop,Hot Dog Joint,Discount Store,Ice Cream Shop


In [184]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 2, chicago_merged.columns[[1] + list(range(5, chicago_merged.shape[1]))]]

,PRIMARY_TYPE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
476,NARCOTICS,2,Bar,Resort,Intersection,Hostel,Fast Food Restaurant,Filipino Restaurant,Exhibit,Eye Doctor,Fabric Shop,Factory


In [185]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 3, chicago_merged.columns[[1] + list(range(5, chicago_merged.shape[1]))]]

,PRIMARY_TYPE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
475,CRIMINAL DAMAGE,3,Intersection,Park,Bus Station,Yoga Studio,Field,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant


In [186]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 4, chicago_merged.columns[[1] + list(range(5, chicago_merged.shape[1]))]]

,PRIMARY_TYPE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
452,BATTERY,4,Park,Fast Food Restaurant,Harbor / Marina,Yoga Studio,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant
453,ARSON,4,Park,Fast Food Restaurant,Harbor / Marina,Yoga Studio,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant
